In [46]:
import boto3
import pandas as pd; pd.set_option('display.max_columns', 200)
import numpy as np
from sklearn.model_selection import GridSearchCV
import tensorflow as tf
from tensorflow import keras

# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import RandomForestClassifier

s3 = boto3.resource('s3')
bucket_name = 'analytics-data-science-competitions'
bucket = s3.Bucket(bucket_name)

## Defining files names
file_key_1 = 'Digit-Recognizer/train.csv'
file_key_2 = 'Digit-Recognizer/test.csv'
file_key_3 = 'Digit-Recognizer/sample_submission.csv'

bucket_object_1 = bucket.Object(file_key_1)
file_object_1 = bucket_object_1.get()
file_content_stream_1 = file_object_1.get('Body')

bucket_object_2 = bucket.Object(file_key_2)
file_object_2 = bucket_object_2.get()
file_content_stream_2 = file_object_2.get('Body')

bucket_object_3 = bucket.Object(file_key_3)
file_object_3 = bucket_object_3.get()
file_content_stream_3 = file_object_3.get('Body')

## Reading data-files
train = pd.read_csv(file_content_stream_1)
test = pd.read_csv(file_content_stream_2)
sample = pd.read_csv(file_content_stream_3)

# Basic Exploration

In [47]:
print('Train dataset dimensions are:', train.shape)

print('Test dataset dimensions are:', test.shape)

Train dataset dimensions are: (42000, 785)
Test dataset dimensions are: (28000, 784)


In [48]:
train['label'].value_counts() / train.shape[0]

1    0.111524
7    0.104786
3    0.103595
9    0.099714
2    0.099452
6    0.098500
0    0.098381
4    0.096952
8    0.096738
5    0.090357
Name: label, dtype: float64

In [49]:
train.columns = [i for i in range(0, 785)]
train = np.array(train)

# Putting the data in the right format

In [56]:
img_rows, img_cols = 28, 28
num_classes = 10

def prep_data(raw):
    out_y = raw[:, 0]
#     out_y = keras.utils.to_categorical(y, num_classes)
    
    x = raw[:,1:]
    num_images = raw.shape[0]
    out_x = x.reshape(num_images, img_rows, img_cols, 1)
    out_x = out_x / 255
    return out_x, out_y

# fashion_file = "../input/fashionmnist/fashion-mnist_train.csv"
# fashion_data = np.loadtxt(fashion_file, skiprows=1, delimiter=',')
x, y = prep_data(train)

# Set up code checking
# from learntools.core import binder
# binder.bind(globals())
# from learntools.deep_learning.exercise_7 import *
# print("Setup Complete")

In [57]:
x.shape

(42000, 28, 28, 1)

In [58]:
y.shape

(42000,)

In [76]:
# Define the model
model = tf.keras.models.Sequential([
        
        tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape = (28, 28, 1)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation = tf.nn.relu),
        tf.keras.layers.Dense(10, activation = tf.nn.softmax)
])

# Setup training parameters
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

# Train the model
print(f'\nMODEL TRAINING:')
model.fit(x, y, epochs = 10)


MODEL TRAINING:
Epoch 1/10
1313/1313 [==============================] - 11s 9ms/step - loss: 0.1739 - accuracy: 0.9469
Epoch 2/10
1313/1313 [==============================] - 11s 9ms/step - loss: 0.0516 - accuracy: 0.9838
Epoch 3/10
1313/1313 [==============================] - 11s 9ms/step - loss: 0.0368 - accuracy: 0.9885
Epoch 4/10
1313/1313 [==============================] - 11s 9ms/step - loss: 0.0275 - accuracy: 0.9908
Epoch 5/10
1313/1313 [==============================] - 11s 9ms/step - loss: 0.0195 - accuracy: 0.9938
Epoch 6/10
1313/1313 [==============================] - 11s 8ms/step - loss: 0.0164 - accuracy: 0.9948
Epoch 7/10
1313/1313 [==============================] - 11s 8ms/step - loss: 0.0127 - accuracy: 0.9960
Epoch 8/10
1313/1313 [==============================] - 11s 9ms/step - loss: 0.0102 - accuracy: 0.9966
Epoch 9/10
1313/1313 [==============================] - 11s 9ms/step - loss: 0.0091 - accuracy: 0.9972
Epoch 10/10
1313/1313 [==============================] -

In [78]:
## Predicting on test
img_rows = 28
img_cols = 28

# test.columns = [i for i in range(0, 784)]
# test = np.array(test)

# num_images = test.shape[0]
# test_images = test.reshape(num_images, img_rows, img_cols, 1)
# test_images = test_images / 255

pred_labels = model.predict(test_images)

In [79]:
np.argmax(pred_labels, axis = 1)

array([2, 0, 9, ..., 3, 9, 2])

In [80]:
data_out = pd.DataFrame({'ImageId': [i for i in range(1, 28001)], 'Label': np.argmax(pred_labels, axis = 1)})
data_out.head()

,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3


In [81]:
data_out.to_csv('submission_2.csv', index = False)

In [11]:
## Defining input and target
X = train.drop(columns = ['label'], axis = 1)
Y = train['label']

## Defining the hyper-parameter grid
logistic_param_grid = {'penalty': ['l1', 'l2', 'elasticnet'],
                       'C': [0.001, 0.01, 0.1, 1, 10, 100],
                       'solver': ['liblinear', 'sag', 'saga']}

## Performing grid search with 5 folds
logistic_grid_search = GridSearchCV(LogisticRegression(), logistic_param_grid, cv = 5, scoring = 'accuracy', n_jobs = -1, verbose = 3).fit(X, Y)

Fitting 5 folds for each of 54 candidates, totalling 270 fits


KeyboardInterrupt: 

In [12]:
28*28

784